# What is this notebook for?
This notebook is intended only for visualinsing the left and or right ventricle segmentations for a particular echonet video.

If you want to add other interesting things to the visualisation (such as estimated septum width, etc.), you should use the [other notebook](./weak_labels.ipynb) instead.

In [10]:
from pathlib import Path
import sys
import math
from typing import List, Tuple

import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import scipy

import echonet
from weak_labels.utils import get_average_eccentricity, get_min_area_rect, get_min_area_box, mask_to_image, image_to_mask, find_corner, get_angle, BOTTOM_LEFT, BOTTOM_RIGHT, remove_septum

config = dotenv_values(".env")

# Can assign these colours to numpy arrays so long as the colours are stored in
# the last axis of the target array (e.g. image.shape =(112, 112, 3), but not
# image.shape = (3, 112, 112)). 
# Just do image[y_vals, x_vals] = MAGENTA
# IMPORTANT: if using within an *opencv* function, you'll want to do 
# COLOUR.tolist() to convert these to python primitives, else opencv complains about
# datatypes
# Note also that these are BGR, not RGB, since that's what opencv prefers BGR for historical reasons!
RED = np.array([0, 0, 255])
GREEN = np.array([0, 255, 0])
BLUE = np.array([255, 0, 0])
ORANGE = np.array([0, 165, 255])
LIGHT_GREY = np.array([211, 211, 211])
MAGENTA = np.array([255, 0, 255])
YELLOW = np.array([0, 255, 255])
WHITE = np.array([255, 255, 255])
BLACK = np.array([0, 0, 0])

# Just some types for us to use in type hints to make dev easier
Point = List[np.intp]
Box = Tuple[Point, Point, Point, Point]
Rectangle = Tuple[Point, Tuple[float, float], float] # [centre, (width, height), angle]

In [11]:
def get_heights(masks: np.ndarray) -> np.ndarray:
    images = mask_to_image(masks)
    heights = np.zeros(len(images))
    for i, image in enumerate(images):
        ((centre_x, centre_y), (width, height), angle) = get_min_area_rect(image)
        height = max(width, height)
        heights[i] = height

    return heights

def get_angles(masks: np.ndarray) -> np.ndarray:
    images = mask_to_image(masks)
    rects = [get_min_area_rect(image) for image in images]
    angles = np.array([get_angle(rect) for rect in rects])
    return angles


def rotate_image(image: np.ndarray, angle: float) -> np.ndarray:
    rect = get_min_area_rect(image)
    bottom_left = find_corner(rect, which=BOTTOM_LEFT)
    bottom_left = (int(bottom_left[0]), int(bottom_left[1]))

    rot_mat = cv2.getRotationMatrix2D(bottom_left, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def get_diastoles_systoles(LV_masks):
    LV_areas = LV_masks.sum(axis=(1,2))
    min_area, max_area = LV_areas.min(), LV_areas.max()
    trim_min = sorted(LV_areas)[round(len(LV_areas) ** 0.05)]
    trim_max = sorted(LV_areas)[round(len(LV_areas) ** 0.95)]
    trim_range = trim_max - trim_min
    diastoles = scipy.signal.find_peaks(LV_areas, distance=20, prominence=(0.50 * trim_range))[0]
    systoles = scipy.signal.find_peaks(-LV_areas, distance=20, prominence=(0.50 * trim_range))[0]

    return diastoles, systoles

def crop_box(image: np.ndarray, box: np.array) -> np.ndarray:
    """
    (num_points, x, y)
    [
        [ 64 464]
        [ 64  64]
        [464  64]
        [464 464]
    ]
    """
    min_x = min(p[0] for p in box)
    max_x = max(p[0] for p in box)
    min_y = min(p[1] for p in box)
    max_y = max(p[1] for p in box)
    return image[min_y:max_y, min_x:max_x]

In [12]:
WRITE = False

config = dotenv_values(".env")

echonet_video_fp = (Path
(config["ECHONET_VIDEO_DIR"]) / config["VIDEONAME"]).with_suffix(".avi")
echonet_video = echonet.utils.loadvideo(str(echonet_video_fp))
echonet_video = echonet_video.transpose((1, 2, 3, 0)) # Put colour axis at end for easier colouring of pixels

LV_masks = np.load(config["LV_MASKS"])
RV_masks = np.load(config["RV_MASKS"])

num_frames, frame_height, frame_width = LV_masks.shape
out_height = (frame_height + frame_height // 8) * 4
out_width = frame_width * 4
out_size = (out_width, out_height)
if WRITE:
    print(f"Size: ({frame_width}, {frame_height}) -> ({out_size})")

WINDOW = f"Segmentation: {config['VIDEONAME']}.avi"
cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)

i = 0
is_playing = True

if WRITE:
    writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'MJPG'), 30, out_size)

try:
    while True:
        if i >= num_frames:
            i = 0
            if WRITE:
                break
        elif i < 0:
            i = num_frames - 1

        # Copy data for this particular frame
        frame = echonet_video[i].copy()
        LV_mask = LV_masks[i].copy()
        RV_mask = RV_masks[i].copy()

        ######## SEGMENTATIONS
        frame[LV_mask] = RED
        frame[RV_mask] = BLUE

        LV_box = get_min_area_box(mask_to_image(LV_mask))
        RV_box = get_min_area_box(mask_to_image(RV_mask))
        cv2.drawContours(frame, [LV_box], 0, YELLOW.tolist())
        cv2.drawContours(frame, [RV_box], 0, ORANGE.tolist())

        ####### ADD FRAME COUNTER AT TOP
        top_border = np.zeros((frame_height // 8, frame_width, 3), dtype=frame.dtype)
        top_border[:, :] = np.expand_dims(LIGHT_GREY, (0, 1))
        cv2.putText(top_border, f"Frame {i+1}/{num_frames}", org=(5,10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.25, color=RED.tolist())
        frame = np.concatenate([top_border, frame], axis=0)

        ######## SHOW SEGMENTATION AND HANDLE KEYPRESS
        cv2.imshow(WINDOW, frame)

        if WRITE:
            frame = cv2.resize(frame, out_size)
            writer.write(frame)

        keypress = cv2.waitKey(25) & 0xFF
        if keypress == ord('q'):
            break
        elif keypress == ord(' '):
            is_playing = not is_playing
        elif keypress == ord('a'):
            i -= 1
        elif keypress == ord('d'):
            i += 1
        else:
            if is_playing:
                i += 1
finally:
    cv2.destroyAllWindows() 
    if WRITE:
        writer.release()